# Sliced solid torus

In this notebook, we show how easy it is to create more complicated structures using set operations with separators, by slicing a torus.

In [1]:
using ValidatedNumerics

In [2]:
solid_torus = @constraint (3 - sqrt(x^2 + y^2))^2 + z^2 <= 1

half_space = @constraint (x + y) + z <= 1

Separator:
  - variables: x, y, z
  - expression: (x + y) + z ∈ [-∞, 1]

In [3]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = pave(solid_torus ∩ half_space, Y, 0.25);

  3.114970 seconds (6.20 M allocations: 256.987 MB, 4.13% gc time)


In [8]:
# @time refine!(paving, 0.1)

  7.595519 seconds (26.86 M allocations: 1.101 GB, 7.49% gc time)


0.1

In [4]:
length(paving.inner)

1994

## 3D visualization

In [5]:
inner = paving.inner
boundary = paving.boundary;

In [6]:
# Pkg.add("GLVisualize")

In [7]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.5/GLVisualize.ji for module GLVisualize.
  likely near /Users/dpsanders/.julia/v0.5/GeometryTypes/src/typealias.jl:90


Set up some colors that depend on the position:

In [18]:
positions = Point{3, Float32}[Point3(mid(x)) for x in vcat(inner, boundary)]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in vcat(inner, boundary)]

zs = Float32[x[3] for x in positions]
minz = minimum(zs)
maxz = maximum(zs)

xs = Float32[x[1] for x in positions]
minx = minimum(xs)
maxx = maximum(xs)

colors1 = RGBA{Float32}[RGBA( (zs[i]-minz)/(maxz-minz), (xs[i]-minx)/(maxx-minx), 0f0, 0.1f0)
        for i in 1:length(inner)]
colors2 = RGBA{Float32}[RGBA( 0.5f0, 0.5f0, 0.5f0, 0.02f0) for x in boundary];
colors = vcat(colors1, colors2);

In [19]:
cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

_view(vis)


@async renderloop(window)

Task (runnable) @0x000000011345ba90